In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

df_clean=pd.read_csv("df_clean_v2.csv", index_col=0)
df_clean.head()

,Platform,Year,Genre,Publisher,Global_Sales,Meta_NP,Meta_VP,Meta_NUsers,Meta_VUsers,test_note_JVC,avis_count_JVC,avis_note_JVC,Classification_Age_JVC,Support_JVC,langue_parlée_JVC,texte_JVC,freq_name
0,NES,1985.0,Platform,Nintendo,40.24,84.0,14.0,8.4,119.0,19.0,3.0,19.0,3.0,Autre,français,français,2.0
1,GB,1989.0,Puzzle,Nintendo,30.26,65.0,10.0,8.0,7.0,11.0,8.0,17.8,3.0,Autre,français,français,4.0
2,Wii,2006.0,Misc,Nintendo,29.02,58.0,41.0,6.6,190.0,12.0,223.0,12.1,3.0,DVD,français,français,4.0
3,DS,2005.0,Simulation,Nintendo,24.76,83.0,54.0,7.6,125.0,15.0,159.0,16.4,3.0,Cartouche,français,français,4.0
4,Wii,2007.0,Sports,Nintendo,22.72,80.0,63.0,7.7,164.0,15.0,227.0,16.2,3.0,DVD,français,français,3.0


In [ ]:
df_gd=pd.get_dummies(df_clean)
df_gd.head()

,Year,Global_Sales,Meta_NP,Meta_VP,Meta_NUsers,Meta_VUsers,test_note_JVC,avis_count_JVC,avis_note_JVC,Classification_Age_JVC,...,Support_JVC_DVD,Support_JVC_DVD Blu-Ray,Support_JVC_PlayStation Store,Support_JVC_UMD,langue_parlée_JVC_anglais,langue_parlée_JVC_français,langue_parlée_JVC_japonais,texte_JVC_anglais,texte_JVC_français,texte_JVC_japonais
0,1985.0,40.24,84.0,14.0,8.4,119.0,19.0,3.0,19.0,3.0,...,0,0,0,0,0,1,0,0,1,0
1,1989.0,30.26,65.0,10.0,8.0,7.0,11.0,8.0,17.8,3.0,...,0,0,0,0,0,1,0,0,1,0
2,2006.0,29.02,58.0,41.0,6.6,190.0,12.0,223.0,12.1,3.0,...,1,0,0,0,0,1,0,0,1,0
3,2005.0,24.76,83.0,54.0,7.6,125.0,15.0,159.0,16.4,3.0,...,0,0,0,0,0,1,0,0,1,0
4,2007.0,22.72,80.0,63.0,7.7,164.0,15.0,227.0,16.2,3.0,...,1,0,0,0,0,1,0,0,1,0


In [ ]:
df_gd.shape

(5837, 356)

In [ ]:
from sklearn.model_selection import train_test_split

y=df_gd['Global_Sales']
X=df_gd.drop('Global_Sales', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.2, random_state=42)


In [ ]:
# TPOT : choix du meilleur modèle de ML
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

# Création de la pipeline
pipeline = make_pipeline(
    StackingEstimator(estimator=DecisionTreeRegressor(max_depth=10, min_samples_leaf=20, min_samples_split=12)),
    VarianceThreshold(threshold=0.1)
)

set_param_recursive(pipeline.steps, 'random_state', 42)

# Entraînement du modèle avec les données d'entraînement
pipeline.fit(X_train, y_train)


In [ ]:
# Pour la régression
from tpot import TPOTRegressor
# Paramétrage du TPOTRegressor
tpot_regression = TPOTRegressor(generations=5, population_size=50, cv=5, scoring='neg_mean_absolute_error', verbosity=2, random_state=42, n_jobs=-1)
 
# Application de TPOT à notre jeu de données train 
tpot_regression.fit(X_train, y_train)

# Calcul du taux de bonnes prédictions
print(tpot_regression.score(X_test, y_test))
# Extraction du code généré par TPOT pour modifier le pipeline créé
tpot_regression.export('tpot_vgsales_2_pipeline.py')

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -0.44772084287597363

Generation 2 - Current best internal CV score: -0.44772084287597363

Generation 3 - Current best internal CV score: -0.44772084287597363

Generation 4 - Current best internal CV score: -0.44772084287597363

Generation 5 - Current best internal CV score: -0.44772084287597363

Best pipeline: ExtraTreesRegressor(RandomForestRegressor(input_matrix, bootstrap=False, max_features=0.25, min_samples_leaf=14, min_samples_split=7, n_estimators=100), bootstrap=False, max_features=0.2, min_samples_leaf=1, min_samples_split=7, n_estimators=100)
-0.42811764126712326


c:\Users\debor\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py:794: FutureWarning: sklearn.metrics.SCORERS is deprecated and will be removed in v1.3. Please use sklearn.metrics.get_scorer_names to get a list of available scorers and sklearn.metrics.get_metric to get scorer.
  warnings.warn(
c:\Users\debor\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
